In [5]:
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
# from matplotlib.colors import ListedColormap

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, confusion_matrix

from sklearn.gaussian_process.kernels import (
WhiteKernel, DotProduct, RBF, ConstantKernel, 
Matern, RationalQuadratic, ExpSineSquared
)

# from sklearn.metrics import PrecisionRecallDisplay

# 1. Import the dataset and separate the label from the features

In [13]:
dataset2 = pd.read_excel('./data/PCOS_data_without_infertility.xlsx', sheet_name="Full_new")
dataset2.drop(columns = 'Unnamed: 44', inplace = True)
dataset2.loc[dataset2['II    beta-HCG(mIU/mL)'] == '1.99.', 'II    beta-HCG(mIU/mL)'] = 1.99
dataset2['II    beta-HCG(mIU/mL)'] = dataset2['II    beta-HCG(mIU/mL)'].astype(float)
dataset2.loc[dataset2['AMH(ng/mL)'] == 'a', 'AMH(ng/mL)'] = np.nan
dataset2['AMH(ng/mL)'] = dataset2['AMH(ng/mL)'].astype(float)
dataset2 = dataset2.dropna()

y = dataset2['PCOS (Y/N)']
features = list(dataset2.columns)
features.remove('Sl. No')
features.remove('Patient File No.')
features.remove('PCOS (Y/N)')
X = dataset2[features].values

# 2. Define models to test: classifiers as we have a binary classification problem

In [15]:
names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    KNeighborsClassifier(3, n_jobs=4),
    SVC(kernel="linear", C=0.025, random_state=42),
    SVC(gamma=2, C=1, random_state=42),
    GaussianProcessClassifier(1.0 * RBF(1.0), random_state=42),
    DecisionTreeClassifier(max_depth=5, random_state=42),
    RandomForestClassifier(
        max_depth=5, n_estimators=10, max_features=1, random_state=42
    ),
    MLPClassifier(alpha=1, max_iter=1000, random_state=42),
    AdaBoostClassifier(algorithm="SAMME", random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

# 3. Calculate accuracy, precision, recall, F1 score

## Note: Scikit-learn's f1-score calculates a different value than what I calculate from the confusion matrix 

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=42
    )

for name, clf in zip(names, classifiers):
    clf = make_pipeline(StandardScaler(), clf)
    clf.fit(X_train, y_train)
    # accuracy
    score = clf.score(X_test, y_test)
    y_pred = clf.predict(X_test)
    # f1 score
    f1 = f1_score(y_test, y_pred)
    # confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    # true positive
    tp = cm[0,0]
    tn = cm[1,1]
    # false positive
    fp = cm[0,1]
    # false negative
    fn = cm[1,0]
    # Precision is the fraction of the correctly classified instances from the total classified instances
    precision = tp / (tp + fp)
    # Recall is the fraction of the correctly classified instances from the total classified instances
    recall = tp / (tp + fn)
    # F1-score
    f1s = 2 * precision * recall / (precision + recall)

    print(name, f1, f1s, 2*tp / (2*tp+fp+fn))




Nearest Neighbors 0.7480916030534351 0.8903654485049834 0.8903654485049833
Linear SVM 0.8175182481751825 0.9152542372881356 0.9152542372881356
RBF SVM 0.0 0.8066298342541436 0.8066298342541437
Gaussian Process 0.8029197080291971 0.9084745762711866 0.9084745762711864
Decision Tree 0.7083333333333334 0.8541666666666666 0.8541666666666666
Random Forest 0.5544554455445545 0.86404833836858 0.8640483383685801
Neural Net 0.8027210884353742 0.8982456140350877 0.8982456140350877
AdaBoost 0.7794117647058824 0.8986486486486487 0.8986486486486487
Naive Bayes 0.6598984771573604 0.7148936170212765 0.7148936170212766
QDA 0.7307692307692307 0.8478260869565217 0.8478260869565217


## Observation: Without hyperparameter optimization, linear SVM, Gaussian process and Neural Net have the highest accuracy and F1-score

In [11]:
from sklearn.model_selection import cross_validate


# Define a function that compares the CV performance of a set of predetermined models 
def cv_comparison(models, X, y, cv):
    # Initiate a DataFrame for the averages and a list for all measures
    cv_metrics = pd.DataFrame(columns = ['Model','Accuracy', 'Precision', 'Recall', 'F1_score', 'ROC_AUC', 'F1', 'DOF', 'Coefficients'])
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    rocaucs = []
    
    # Loop through the models, run a CV, add the average scores to the DataFrame and the scores of 
    # all CVs to the list
    for model in models:
        # calculate model complexity
        number_of_model_coeffs = 0
        try:
            for i in model.coef_:
                if i != 0:
                    number_of_model_coeffs +=1
            coefficients = model.coef_
        except:
            number_of_model_coeffs = np.nan
            coefficients = np.nan
        
        # get cross-validation metrics
        cv_results = cross_validate(model, X, y, 
                                    scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc'], 
                                    cv=cv,n_jobs=4)
        
        # save metrics
        accuracy = np.round(cv_results['test_accuracy'].mean(),4)
        accuracies.append(cv_results['test_accuracy'])
        precision = np.round(cv_results['test_precision'].mean(),4)
        precisions.append(cv_results['test_precision'])
        recall = np.round(cv_results['test_recall'].mean(),4)
        recalls.append(cv_results['test_recall'])
        f1 = np.round(cv_results['test_f1'].mean(),4)
        f1_scores.append(cv_results['test_f1'])
        rocauc = np.round(cv_results['test_roc_auc'].mean(),4)
        rocaucs.append(cv_results['test_roc_auc'])
        # calculate F1 score again, as scikit learn function had weird results above 
        # checked and doesn't make a difference
        # f1s = 2 * precision * recall / (precision + recall)
        # summary dataframe
        df = pd.DataFrame(data = {'Model': str(model),
                                  'Accuracy': accuracy,
                                  'Precision': precision,
                                  'Recall': recall,
                                  'F1_score': f1,
                                  'ROC_AUC': rocauc,
                                #   'F1': f1s,
                                  'DOF': number_of_model_coeffs,
                                  'Coefficients': coefficients}, 
                                  index = [cv_metrics.index.max()+1])
        cv_metrics = pd.concat([cv_metrics, df], axis=0, ignore_index = True)

    return cv_metrics, accuracies, precisions, recalls, f1_scores, rocaucs

# 4. Cross-validation, 5-fold

In [106]:
# Create the models to be tested
# mlr_reg = LinearRegression()
# ard_reg=ARDRegression(compute_score=True)
# #rf_reg = RandomForestRegressor(random_state=42)
# #xgb_reg = XGBRegressor(random_state=42)
# Put the models in a list to be used for Cross-Validation
models = classifiers

# def ridge_model(alpha):
#     ridgereg = Ridge(alpha=alpha, max_iter=int(1e6))
#     return ridgereg

# def lasso_model(alpha):
#     #Fit the model
#     lassoreg = Lasso(alpha=alpha, max_iter=int(1e6))
#     return lassoreg

#Define the alpha values to test
# alpha_lasso = [0.0001, 0.001, 0.01, 0.02, 0.03, 0.04, 0.05,0.075, 0.1,0.15, 0.2, 0.25, 0.5, 0.75, 1] 
# alpha_ridge = [1, 3, 4, 5, 6, 7, 10, 15, 20, 30, 50]

# for a in alpha_lasso:
#     models.append(lasso_model(a))
# for a in alpha_ridge:    
#     models.append(ridge_model(a))

# Run the Cross-Validation comparison with the models used in this analysis
# cv = n - leave one out cross validation
comp, accuracies, precisions, recalls, f1_scores, rocaucs = cv_comparison(models, StandardScaler().fit_transform(X_train), y_train, 5)

/Users/szilvia/.pyenv/versions/python-3.8.5/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/szilvia/.pyenv/versions/python-3.8.5/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/szilvia/.pyenv/versions/python-3.8.5/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/szilvia/.pyenv/versions/

## Observation: After cross-validation, MLP Classifier and ADABoost are the best models overall (based on F1-score)

In [103]:
comp

,Model,Accuracy,Precision,Recall,F1_score,ROC_AUC,F1,DOF,Coefficients
0,KNeighborsClassifier(n_neighbors=3),0.8448,0.8063,0.7074,0.7494,0.8826,0.753619,NaN,NaN
1,"SVC(C=0.025, kernel='linear', random_state=42)",0.8758,0.8696,0.7359,0.7934,0.9511,0.797183,NaN,NaN
2,"SVC(C=1, gamma=2, random_state=42)",0.6708,0.0000,0.0000,0.0000,0.6192,NaN,NaN,NaN
3,GaussianProcessClassifier(kernel=1**2 * RBF(le...,0.8444,0.7053,0.6117,0.6544,0.9412,0.655174,NaN,NaN
4,"DecisionTreeClassifier(max_depth=5, random_sta...",0.8320,0.7538,0.7346,0.7408,0.7937,0.744076,NaN,NaN
5,"RandomForestClassifier(max_depth=5, max_featur...",0.7952,0.8464,0.4429,0.5751,0.8886,0.581510,NaN,NaN
6,"MLPClassifier(alpha=1, max_iter=1000, random_s...",0.8819,0.8396,0.8117,0.8203,0.9476,0.825414,NaN,NaN
7,"AdaBoostClassifier(algorithm='SAMME', random_s...",0.8789,0.8103,0.8307,0.8190,0.9491,0.820373,NaN,NaN
8,GaussianNB(),0.6989,0.5733,0.9056,0.6806,0.9031,0.702117,NaN,NaN
9,QuadraticDiscriminantAnalysis(),0.8106,0.7259,0.6896,0.7042,0.8058,0.707285,NaN,NaN


In [113]:
f1_scores

[array([0.7       , 0.7804878 , 0.74285714, 0.71428571, 0.80952381]),
 array([0.75675676, 0.82051282, 0.78947368, 0.73684211, 0.86363636]),
 array([0., 0., 0., 0., 0.]),
 array([0.8       , 0.87179487, 0.85      , 0.75      , 0.        ]),
 array([0.85      , 0.80851064, 0.57894737, 0.8       , 0.66666667]),
 array([0.28571429, 0.66666667, 0.64516129, 0.5       , 0.77777778]),
 array([0.7804878 , 0.87179487, 0.85      , 0.79069767, 0.80851064]),
 array([0.82926829, 0.79069767, 0.9047619 , 0.74418605, 0.82608696]),
 array([0.6779661 , 0.63492063, 0.82051282, 0.55555556, 0.71428571]),
 array([0.7       , 0.66666667, 0.7       , 0.63636364, 0.81818182])]

In [84]:
# What kind of scoring does scikit-learn provide?
from sklearn.metrics import SCORERS
sorted(SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

# 5. Hyperparameter tuning

In [71]:

def neighborsclass(n,l):
    return KNeighborsClassifier(n_neighbors=n,leaf_size=l, n_jobs=4)

def decisiontree(criterion, depth, ccp_alpha):
    return DecisionTreeClassifier(criterion=criterion, max_depth=depth, ccp_alpha=ccp_alpha,random_state=42)

def randomforest(max_depth, n_estimators, max_features):
    return RandomForestClassifier(max_depth=max_depth, 
                                  n_estimators=n_estimators, 
                                  max_features=max_features, 
                                  n_jobs=4, 
                                  random_state=42)

def adaboost(n_estimators):
    return AdaBoostClassifier(n_estimators=n_estimators, algorithm="SAMME", random_state=42)

def mlpc(hidden_layer_sizes, alpha):
    return MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, alpha=alpha, max_iter=1000, random_state=42)

# Define a function that compares the CV performance of a set of predetermined models 
def cv_comparison2(models, X, y, cv):
    # Initiate a DataFrame for the averages and a list for all measures
    cv_metrics = pd.DataFrame(columns = ['Model','Accuracy', 'Precision', 'Recall', 'F1_score', 'ROC_AUC'])
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    rocaucs = []
    
    # Loop through the models, run a CV, add the average scores to the DataFrame and the scores of 
    # all CVs to the list
    for model in models:
        # get cross-validation metrics
        cv_results = cross_validate(model, X, y, 
                                    scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc'], 
                                    cv=cv,n_jobs=4)
        
        # save metrics
        accuracy = np.round(cv_results['test_accuracy'].mean(),4)
        accuracies.append(cv_results['test_accuracy'])
        precision = np.round(cv_results['test_precision'].mean(),4)
        precisions.append(cv_results['test_precision'])
        recall = np.round(cv_results['test_recall'].mean(),4)
        recalls.append(cv_results['test_recall'])
        f1 = np.round(cv_results['test_f1'].mean(),4)
        f1_scores.append(cv_results['test_f1'])
        rocauc = np.round(cv_results['test_roc_auc'].mean(),4)
        rocaucs.append(cv_results['test_roc_auc'])
        # calculate F1 score again, as scikit learn function had weird results above 
        # checked and doesn't make a difference
        # f1s = 2 * precision * recall / (precision + recall)
        # summary dataframe
        df = pd.DataFrame(data = {'Model': str(model),
                                  'Accuracy': accuracy,
                                  'Precision': precision,
                                  'Recall': recall,
                                  'F1_score': f1,
                                  'ROC_AUC': rocauc
                                  }, 
                                  index = [cv_metrics.index.max()+1])
        cv_metrics = pd.concat([cv_metrics, df], axis=0, ignore_index = True)

    return cv_metrics, accuracies, precisions, recalls, f1_scores, rocaucs

In [72]:
models = []
# Gaussian Naive Bayes
# no priors => no hyperparameters
models.append(GaussianNB())
# QDA
# no priors => no hyperparameters
models.append(QuadraticDiscriminantAnalysis())
# Gaussian process
# different kernels (and optimizers)
for kernel in [1.0 * RBF(1.0),
               RBF() + ConstantKernel(constant_value=2),
               DotProduct() + WhiteKernel(noise_level=0.5),
               DotProduct() + WhiteKernel(), # dot product kernel is invariant to a rotation of the coordinates about the origin, but not translations
            #    DotProduct(sigma_0=0) + WhiteKernel(), #homogeneous
               1.0 * Matern(length_scale=1.0, nu=1.5), # nu is smoothness of Kernel
               1.0 * Matern(length_scale=1.0, nu=2),
               1.0 * Matern(length_scale=1.0, nu=2.5),
               RationalQuadratic(length_scale=1.0, alpha=1.5),
               RationalQuadratic(length_scale=1.0, alpha=5),
               ExpSineSquared(length_scale=1, periodicity=1,) # periodic kernel
               ]:
    models.append(GaussianProcessClassifier(kernel=kernel, random_state=42, n_jobs=4))
# KNearest neighbor
for n in [2,3,4,5,7,10]:
    # leaf size doesn't have a lot of effect on this dataset
    # could remove this inner loop to decrease number of models
    # for l in [10, 30, 100]:
        # models.append(neighborsclass(n,l))
    models.append(neighborsclass(n,30))
# decision tree
for criterion in ['gini', 'entropy']:
    for depth in [2, 5, 10, None]:
        for ccp_alpha in [0, 2, 5, 10]:
            models.append(decisiontree(criterion, depth, ccp_alpha))
# random forest
for max_depth in [3,5,10]: #maximum depth of the tree
    for n_estimators in [10, 20, 40, 80]: # number of trees in forest
         for max_features in ["sqrt", "log2"]: # number of features to consider when looking for the best split
            models.append(randomforest(max_depth, n_estimators, max_features))
# AdaBoost
for n_estimators in [5, 10, 20, 50, 100]:
    models.append(adaboost(n_estimators))
# Multilayer Perceptron
for hidden_layer_sizes in [(50,),(100,), (200,)]:
    for alpha in [1e-5, 1e-3, 1, 10]: #L2 regularization strength
        models.append(mlpc(hidden_layer_sizes, alpha))


In [73]:
len(models)

91

In [74]:
comp, accuracies, precisions, recalls, f1_scores, rocaucs = cv_comparison2(models, StandardScaler().fit_transform(X_train), y_train, 5)

/Users/szilvia/.pyenv/versions/python-3.8.5/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/szilvia/.pyenv/versions/python-3.8.5/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [ ]:
comp.sort_values(by=['ROC_AUC', 'F1_score'], ascending=False)

,Model,Accuracy,Precision,Recall,F1_score,ROC_AUC
9,GaussianProcessClassifier(kernel=RationalQuadr...,0.6708,0.0000,0.0000,0.0000,0.9605
10,GaussianProcessClassifier(kernel=RationalQuadr...,0.8758,0.9052,0.6978,0.7864,0.9567
3,GaussianProcessClassifier(kernel=RBF(length_sc...,0.8726,0.9023,0.6883,0.7793,0.9567
6,GaussianProcessClassifier(kernel=1**2 * Matern...,0.8850,0.8574,0.7926,0.8205,0.9527
7,GaussianProcessClassifier(kernel=1**2 * Matern...,0.8819,0.8553,0.7831,0.8144,0.9520
8,GaussianProcessClassifier(kernel=1**2 * Matern...,0.8788,0.8493,0.7831,0.8109,0.9514
4,GaussianProcessClassifier(kernel=DotProduct(si...,0.8818,0.8408,0.8013,0.8180,0.9507
5,GaussianProcessClassifier(kernel=DotProduct(si...,0.8818,0.8408,0.8013,0.8180,0.9507
2,GaussianProcessClassifier(kernel=1**2 * RBF(le...,0.8444,0.7053,0.6117,0.6544,0.9412
17,"KNeighborsClassifier(n_jobs=4, n_neighbors=10)",0.8663,0.9213,0.6506,0.7616,0.9400
